In [ ]:
# default_exp core

# Hello nbdev

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

## Functions

In [ ]:
#export
def say_hello(to):
    "Say hello to somebody"
    return(f'Hello {to}!')

We can use this to say hello to someone.

In [ ]:
say_hello("Sylvain")

'Hello Sylvain!'

In [ ]:
test_eq(say_hello("Jeremy"), "Hello Jeremy!")

In [ ]:
#export
class HelloSayer:
    "Say hello to `to` using `say_hello`"
    def __init__(self, to): self.to = to
    def say(self):
        "do the saying"
        return say_hello(self.to)

In [ ]:
show_doc(HelloSayer.say)

<h4 id="HelloSayer.say" class="doc_header"><code>HelloSayer.say</code><a href="__main__.py#L5" class="source_link" style="float:right">[source]</a></h4>

> <code>HelloSayer.say</code>()

do the saying

In [ ]:
o = HelloSayer("Bichec")
o.say()

'Hello Bichec!'